In [1]:
import pandas as pd
import numpy as np

In [22]:
df = pd.read_csv('../data/estc_processed.csv')

/home/konstaku/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
list(df.columns)

['control_number',
 'language.English',
 'language.French',
 'language.Latin',
 'language.German',
 'language.Scottish Gaelic',
 'language.Italian',
 'language.Greek Ancient to 1453 ',
 'language.Welsh',
 'language.Portuguese',
 'language.Dutch',
 'language.Greek Modern 1453- ',
 'language.Hebrew',
 'language.Spanish',
 'language.Pahlavi',
 'language.Swedish',
 'language.Irish',
 'language.Manx',
 'language.Romance Other ',
 'language.Algonquian Other ',
 'language.Lithuanian',
 'language.Turkish',
 'language.English Old ca. 450-1100 ',
 'language.Scots',
 'language.Arabic',
 'language.North American Indian Other ',
 'language.Persian',
 'language.French Middle ca. 1300-1600 ',
 'language.Newari',
 'language.Armenian',
 'language.Tamil',
 'language.Icelandic',
 'language.Bengali',
 'language.Russian',
 'language.Malayalam',
 'language.Danish',
 'language.English Middle 1100-1500 ',
 'language.Coptic',
 'language.Mongolian',
 'language.Gujarati',
 'language.Malay',
 'language.Sanskrit',

In [24]:
columns = np.array([
    'control_number', 'system_control_number', 'author_name', 'author_birth', 'author_death', 'title', 'publication_year_from',
 'publication_year_till',
 'volnumber',
 'volcount',
 'parts',
 'publisher',
 'publication_place',
 'author_pseudonyme',
 'author',
 'publication_year',
 'publication_decade',
 'id',
])

df2 = df[columns]
df2 = df2.set_index('system_control_number')

In [25]:
df_canon = pd.read_csv('../data/estc_metadata_work_id_and_canon_status.csv')

In [26]:
df_canon = df_canon[df_canon['is_canon']== True]
df_canon = df_canon.set_index('system_control_number')
df_canon = df_canon.join(df2)

In [27]:
df_canon.columns

Index(['Unnamed: 0', 'work_id', 'is_canon', 'control_number', 'author_name',
       'author_birth', 'author_death', 'title', 'publication_year_from',
       'publication_year_till', 'volnumber', 'volcount', 'parts', 'publisher',
       'publication_place', 'author_pseudonyme', 'author', 'publication_year',
       'publication_decade', 'id'],
      dtype='object')

In [28]:
# Create texts
df_final = df_canon[['id', 'title']]
df_final.to_csv('../data/texts.csv')
text_ids = set(list(df_canon['id']))

In [18]:
# Create actors
df_actors = pd.read_csv('../data/unified_actors.csv')
df_actor_details = pd.read_csv('../data/unified_actorlinks_enriched.csv')
df_actor_details = df_actor_details.set_index('curives')

actor_ids = set(df_actors['actor_id'])
df_actors = df_actors[['actor_id', 'name_unified']]
names_by_id = df_actor_details[['actor_id', 'actor_name_primary']].set_index('actor_id').to_dict()['actor_name_primary']

def get_name(val):
    return names_by_id_df.get(val, 'N/A')

df_actors['name'] = df_actors['actor_id'].apply(get_name)
df_actors = df_actors.set_index('actor_id')[['name']]
df_actors.to_csv('../data/actors.csv')

In [45]:
df_canon_join = df_canon[['id']].join(df_actor_details)[['actor_id', 'id', 'actor_role_author', 'actor_role_printer', 'actor_role_publisher', 'actor_role_bookseller', 'actor_role_corporate_author', 'actor_role_geographic_record', 'actor_role_unknown', 'actor_role_translator', 'actor_role_editor', 'actor_role_engraver', 'actor_role_other', 'pubyear', 'publoc']]

In [127]:
entity_connections = []
processed_ids = set([])
years, places = set([]), set([])
year_connections = []
place_connections = []

roles = ['actor_role_author', 'actor_role_printer', 'actor_role_publisher', 'actor_role_bookseller', 'actor_role_corporate_author', 'actor_role_geographic_record', 'actor_role_unknown', 'actor_role_translator', 'actor_role_editor', 'actor_role_engraver', 'actor_role_other'] 
connections = ['author', 'printer', 'publisher', 'bookseller', 'corporate author', 'geographic record', 'unknown' , 'translator', 'editor', 'engraver', 'other']
stats = ['pubyear', 'publoc']

def process_connections(row):
    for role, connection in zip(roles, connections):
        if row[role]:
            entity_connections.append({'text': row['id'], 'entity': row['actor_id'], 'type': connection})
        if row['id'] not in processed_ids:
            processed_ids.add(row['id'])
            if row.get('publoc'):
                places.add(row['publoc'])
                place_connections.append({'text': row['id'], 'place': row['publoc']})
            if row.get('pubyear'):
                years.add(row['pubyear'])
                year_connections.append({'text': row['id'], 'year': row['pubyear']})    
            
for i, (_, row) in enumerate(df_canon_join.iterrows()):
    
    if i % 10000 == 5000:
        print(f'Progress at {i} / {len(df_canon_join)}')
    process_connections(row)

Progress at 5000 / 112065
Progress at 15000 / 112065
Progress at 25000 / 112065
Progress at 35000 / 112065
Progress at 45000 / 112065
Progress at 55000 / 112065
Progress at 65000 / 112065
Progress at 75000 / 112065
Progress at 85000 / 112065
Progress at 95000 / 112065
Progress at 105000 / 112065


In [123]:
year_df = pd.DataFrame.from_dict(year_connections).set_index('text')
place_df = pd.DataFrame.from_dict(place_connections).set_index('text')
entity_df = pd.DataFrame.from_dict(entity_connections).set_index('text')

year_df.to_csv('../data/year_connections.csv')
place_df.to_csv('../data/place_connections.csv')
entity_df.to_csv('../data/entity_connections.csv')

In [177]:
text_pairs = pd.read_csv('../data/idpairs.csv')
pair_by_fulltext_id = text_pairs.set_index('fulltext_id').to_dict()['curives_id']


In [183]:
fulltext_ids = (set([str(x) for x in text_pairs['fulltext_id']]))


In [189]:
import json
import time
import os
clusters = {}
json_files = [c for c in os.listdir('../data/clusters') if not c.endswith('.gz') and 'ipynb_checkpoints' not in c]

np.random.shuffle(json_files)
start = time.time()
for i, json_path in enumerate(json_files):
    with open('../data/clusters/' + json_path) as f:
        data = json.load(f)
        for cluster, reuses in data.items():
            reuse_arr = []
            for entry in reuses.get('hits', []):
                id_ref = entry['doc_id'].split('/')[-1].split('.')[0]
                if id_ref in fulltext_ids:
                    try:
                        curive = pair_by_fulltext_id[int(id_ref)]
                        if curive in text_ids:
                            reuse_arr.append({'text_id': id_ref, 'reuse_id': entry['id'], 'curives_id': curive})
                    except:
                        pass
                        
                    
            if len(reuse_arr) > 0:
                clusters[cluster] = reuse_arr
    
    if i % 1000 == 999:
        print(f'Time for last 1000 cluster files: {time.time() - start}' f' Progress: {i + 1} / {len(json_files)}')
        start = time.time()

Time for last 1000 cluster files: 63.817973136901855 Progress: 1000 / 28319
Time for last 1000 cluster files: 65.22922039031982 Progress: 2000 / 28319
Time for last 1000 cluster files: 63.60137438774109 Progress: 3000 / 28319
Time for last 1000 cluster files: 61.56176710128784 Progress: 4000 / 28319
Time for last 1000 cluster files: 61.720980644226074 Progress: 5000 / 28319
Time for last 1000 cluster files: 58.04584050178528 Progress: 6000 / 28319
Time for last 1000 cluster files: 60.80421209335327 Progress: 7000 / 28319
Time for last 1000 cluster files: 59.57559275627136 Progress: 8000 / 28319
Time for last 1000 cluster files: 59.610456705093384 Progress: 9000 / 28319
Time for last 1000 cluster files: 63.264538526535034 Progress: 10000 / 28319
Time for last 1000 cluster files: 58.29667663574219 Progress: 11000 / 28319
Time for last 1000 cluster files: 58.213151931762695 Progress: 12000 / 28319
Time for last 1000 cluster files: 59.76942753791809 Progress: 13000 / 28319
Time for last 10

In [250]:
import time

year_by_id = year_df.to_dict()['year']

reuse_connections = []

start = time.time()
def create_connections(cluster, cid):
    if len(cluster) == 1:
        return
    
    cluster = list(filter(lambda c: year_by_id[c['curives_id']] == year_by_id[c['curives_id']], cluster))
    cs = sorted([{'fulltext_id': c['text_id'], 'text_id': c['curives_id'], 'year': int(year_by_id[c['curives_id']])} for c in cluster], key=lambda x: -x['year'])
    
    for i in range(len(cs) - 1):
        for j in range(i + 1, len(cs)):
            start = cs[i]
            end = cs[j]
            reuse_connections.append({'from': start['text_id'], 'to': end['text_id'], 'cluster': cid})

for i, (k, v) in enumerate(clusters.items()):
    create_connections(v, k)
    if i % 1_000_000 == 500_000:
        print(f'Time for last 1_000_000 cluster files: {time.time() - start}' f' Progress: {i + 1} / {len(clusters)}')
        start = time.time()
    

Time for last 1_000_000 cluster files: 3.3575356006622314 Progress: 500001 / 4964834
Time for last 1_000_000 cluster files: 61.51358246803284 Progress: 1500001 / 4964834
Time for last 1_000_000 cluster files: 7.419744491577148 Progress: 2500001 / 4964834
Time for last 1_000_000 cluster files: 7.536660671234131 Progress: 3500001 / 4964834
Time for last 1_000_000 cluster files: 10.398154020309448 Progress: 4500001 / 4964834


In [253]:
cluster_df = pd.DataFrame.from_dict(reuse_connections).set_index('cluster')
cluster_df.to_csv('../data/cluster_connections.csv')

In [10]:
count = 0
batch = 666_666
while count < len(df):
    df[count:count+batch].set_index('cluster').to_csv(f'../data/cluster_connections-{count // batch}.csv')
    count += batch

In [263]:
len(reuse_connections)

39586582

In [2]:
import pandas as pd
df = pd.read_csv('../data/cluster_connections.csv')

In [5]:
(df[:500_000]).set_index('cluster').to_csv(f'../data/cluster_connectionstest.csv')

In [13]:
len(df[df['from'] == df['to'] ])

544724

In [14]:
len(df)

39586582